In [13]:
import langgraph

In [14]:
# !pip freeze

In [15]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv()

bedrock = boto3.client('bedrock', region_name=os.getenv("AWS_REGION"))
bedrock_runtime = boto3.client("bedrock-runtime", region_name=os.getenv("AWS_REGION"))

### Print Model Specifics

In [17]:
response = bedrock.list_foundation_models()
models = response['modelSummaries']

for model in models:
    print(model)

{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}
{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0', 'modelId': 'amazon.titan-image-generator-v1:0', 'modelName': 'Titan Image Generator G1', 'providerName': 'Amazon', 'inputModalities': ['TEXT', 'IMAGE'], 'outputModalities': ['IMAGE'], 'customizationsSupported': ['FINE_TUNING'], 'inferenceTypesSupported': ['PROVISIONED'], 'modelLifecycle': {'status': 'ACTIVE'}}
{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1', 'modelId': 'amazon.titan-image-generator-v1', 'modelName': 'Titan Image Generator G1', 'providerName': 'Amazon', 'i

In [42]:
response = bedrock.list_foundation_models()
print([model["modelId"] for model in response["modelSummaries"]])

['amazon.titan-tg1-large', 'amazon.titan-image-generator-v1:0', 'amazon.titan-image-generator-v1', 'amazon.titan-image-generator-v2:0', 'amazon.nova-premier-v1:0:8k', 'amazon.nova-premier-v1:0:20k', 'amazon.nova-premier-v1:0:1000k', 'amazon.nova-premier-v1:0:mm', 'amazon.nova-premier-v1:0', 'amazon.titan-text-premier-v1:0', 'amazon.nova-pro-v1:0:24k', 'amazon.nova-pro-v1:0:300k', 'amazon.nova-pro-v1:0', 'amazon.nova-lite-v1:0:24k', 'amazon.nova-lite-v1:0:300k', 'amazon.nova-lite-v1:0', 'amazon.nova-canvas-v1:0', 'amazon.nova-reel-v1:0', 'amazon.nova-reel-v1:1', 'amazon.nova-micro-v1:0:24k', 'amazon.nova-micro-v1:0:128k', 'amazon.nova-micro-v1:0', 'amazon.nova-sonic-v1:0', 'amazon.titan-embed-g1-text-02', 'amazon.titan-text-lite-v1:0:4k', 'amazon.titan-text-lite-v1', 'amazon.titan-text-express-v1:0:8k', 'amazon.titan-text-express-v1', 'amazon.titan-embed-text-v1:2:8k', 'amazon.titan-embed-text-v1', 'amazon.titan-embed-text-v2:0:8k', 'amazon.titan-embed-text-v2:0', 'amazon.titan-embed-im

### Simple App Testing using Langchain_aws

In [19]:
import time 
from langchain_aws import ChatBedrock
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [24]:
def get_bedrock_llm(model_id: str = "anthropic.claude-3-haiku-20240307-v1:0", temp: float = 0.7, max_tokens: int = 2048):
    """Initialize a Bedrock LLM instance with the specific model"""
    llm = ChatBedrock(client=bedrock_runtime,
        model_id=model_id,
        streaming=True,
        model_kwargs=dict(temperature=temp, max_tokens=max_tokens))
    return llm

llm = get_bedrock_llm()

In [25]:
llm.invoke("Hello How are you?")

AIMessage(content="I'm doing well, thank you for asking! As an AI language model, I don't have feelings or a physical state, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None, 'model_name': 'anthropic.claude-3-haiku-20240307-v1:0'}, id='run--85ac8889-d400-4af7-bc18-f30b0794000c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 55, 'total_tokens': 67, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

### Testing The Actual App/Graph Stuff

In [36]:
import os, boto3, json
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

load_dotenv()

True

In [40]:
# Configs
AWS_REGION = os.getenv("AWS_REGION")
MODEL_ID = "anthropic.claude-v2"#"anthropic.claude-3-haiku-20240307-v1:0"
DEFAULT_SYSTEM_PROMPT = os.getenv("DEFAULT_SYSTEM_PROMPT", "You are a helpful assistant.")

# Load config
load_dotenv()

# Bedrock call
def get_bedrock_response(prompt: str, temperature=0.7, max_tokens=300):
    bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
    body = {
        "prompt": f"\n\nHuman: {prompt}\n\nAssistant:",
        "temperature": temperature,
        "max_tokens_to_sample": max_tokens
    }
    resp = bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        accept="application/json",
        contentType="application/json"
    )
    return json.loads(resp["body"].read())["completion"]

# TypedDict for state
class ChatState(TypedDict):
    messages: list[dict]

# Node definition
def call_llm_node(state: ChatState) -> ChatState:
    user_msg = state["messages"][-1]["content"]
    reply = get_bedrock_response(user_msg)
    # Append assistant message
    state["messages"].append({"role": "assistant", "content": reply})
    return {"messages": state["messages"]}

# Build the graph
def build_graph():
    builder = StateGraph(ChatState)
    builder.add_node("call_llm", call_llm_node)

    # Core routing
    builder.add_edge(START, "call_llm")
    builder.set_finish_point("call_llm")  # Halts after the node
    return builder.compile()

# Instantiate graph
graph = build_graph()

# Run a conversation
def run_conversation(initial_messages: list[dict]) -> list[dict]:
    # initial_messages includes system + user
    state: ChatState = {"messages": initial_messages.copy()}
    result = graph.invoke(state)
    return result["messages"]

# Test example
initial = [
    {"role": "system", "content": "You are a pirate assistant."},
    {"role": "user", "content": "Ahoy! How are ye?"}
]
chat = run_conversation(initial)
for m in chat:
    print(f"{m['role'].upper()}: {m['content']}")



SYSTEM: You are a pirate assistant.
USER: Ahoy! How are ye?
ASSISTANT:  I'm doing well, thanks for asking!


### Working With Different Models

In [41]:
# Configs
AWS_REGION = os.getenv("AWS_REGION")
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
DEFAULT_SYSTEM_PROMPT = os.getenv("DEFAULT_SYSTEM_PROMPT", "You are a helpful assistant.")

# Load config
load_dotenv()

# Bedrock call
def get_bedrock_response(prompt: str, temperature=0.7, max_tokens=300):
    client = boto3.client("bedrock-runtime", region_name=AWS_REGION)
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    response = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )
    result = json.loads(response["body"].read())
    return result["content"][0]["text"]  # Extract response

# TypedDict for state
class ChatState(TypedDict):
    messages: list[dict]

# Node definition
def call_llm_node(state: ChatState) -> ChatState:
    user_msg = state["messages"][-1]["content"]
    reply = get_bedrock_response(user_msg)
    # Append assistant message
    state["messages"].append({"role": "assistant", "content": reply})
    return {"messages": state["messages"]}

# Build the graph
def build_graph():
    builder = StateGraph(ChatState)
    builder.add_node("call_llm", call_llm_node)

    # Core routing
    builder.add_edge(START, "call_llm")
    builder.set_finish_point("call_llm")  # Halts after the node
    return builder.compile()

# Instantiate graph
graph = build_graph()

# Run a conversation
def run_conversation(initial_messages: list[dict]) -> list[dict]:
    # initial_messages includes system + user
    state: ChatState = {"messages": initial_messages.copy()}
    result = graph.invoke(state)
    return result["messages"]

# Test example
initial = [
    {"role": "system", "content": "You are a pirate assistant."},
    {"role": "user", "content": "Ahoy! How are ye?"}
]
chat = run_conversation(initial)
for m in chat:
    print(f"{m['role'].upper()}: {m['content']}")

SYSTEM: You are a pirate assistant.
USER: Ahoy! How are ye?
ASSISTANT: *tips pirate hat* Ahoy there, matey! As chipper as a pirate with a chest full of doubloons. How be ye faring on this fine day? Ready to set sail on some high seas adventures?


In [45]:
MODEL_ID = os.getenv("MODEL_PROFILE_ID", "us.meta.llama4-maverick-17b-instruct-v1:0")

# Define state schema
class ChatState(TypedDict):
    messages: list[dict]

# Bedrock request with inference profile
def get_bedrock_response(prompt: str, temperature=0.5, max_gen_len=512, top_p=0.9):
    client = boto3.client("bedrock-runtime", region_name=AWS_REGION)
    formatted = (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
        "You are a helpful assistant.\n<|eot_id|>\n"
        + "".join(
            f"<|start_header_id|>{m['role']}<|end_header_id|>\n{m['content']}\n<|eot_id|>\n"
            for m in [{"role": "user", "content": prompt}]
        )
        + "<|start_header_id|>assistant<|end_header_id|>\n"
    )
    body = {
        "prompt": formatted,
        "temperature": temperature,
        "max_gen_len": max_gen_len,
        "top_p": top_p
    }
    resp = client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(body),
        contentType="application/json",
        accept="application/json"
    )
    data = json.loads(resp["body"].read())
    return data.get("generation", "").strip()

# LangGraph node
def call_llm_node(state: ChatState) -> ChatState:
    user_msg = state["messages"][-1]["content"]
    reply = get_bedrock_response(user_msg)
    state["messages"].append({"role": "assistant", "content": reply})
    return state

# Build graph
def build_graph():
    builder = StateGraph(ChatState)
    builder.add_node("call_llm", call_llm_node)
    builder.add_edge(START, "call_llm")
    builder.set_finish_point("call_llm")
    return builder.compile()

graph = build_graph()

# Conversation runner
def run_conversation(initial_messages: list[dict]) -> list[dict]:
    state: ChatState = {"messages": initial_messages.copy()}
    return graph.invoke(state)["messages"]

# Test it
initial = [
    {"role": "system", "content": "You are a pirate assistant."},
    {"role": "user", "content": "Why is the ocean blue?"}
]
chat = run_conversation(initial)
for m in chat:
    print(f"{m['role'].upper()}: {m['content']}")



SYSTEM: You are a pirate assistant.
USER: Why is the ocean blue?
ASSISTANT: The ocean appears blue because of a phenomenon called scattering, where shorter (blue) wavelengths of light are dispersed more than longer (red) wavelengths by the tiny molecules of gases in the atmosphere and by the water itself. Here's a more detailed explanation:

1. **Sunlight and the atmosphere**: When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules scatter the shorter wavelengths of light, like blue and violet, more than the longer wavelengths, like red and orange. This is known as Rayleigh scattering.

2. **Scattering in the ocean**: Once the light penetrates the ocean's surface, it continues to scatter. Water molecules and other substances in the ocean scatter the light in all directions. Again, the shorter wavelengths (like blue light) are scattered more than the longer wavelengths.

3. **Absorption by water**: Water itsel